In [1]:
#  Import a bunch of stuff
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'hot'
import IPython.display as ipd

import audio_functions as af
import model_functions as mf
import dataset
import librosa
import soundfile as sf
import tensorflow as tf
from keras import layers
from SegCaps import capsule_layers
from functools import partial
import datetime

Using TensorFlow backend.


In [23]:
#  Set some variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=4
normalise=False
batch_size = 1
shuffle=False
n_shuffle = 1
mag_phase = False

In [24]:
def get_paired_dataset(zipped_files,
                       sample_rate,
                       n_fft,
                       fft_hop,
                       patch_window,
                       patch_hop,
                       n_parallel_readers,
                       batch_size,
                       n_shuffle,
                       normalise):

    return (
        tf.data.Dataset.from_tensor_slices((zipped_files[:, 0], zipped_files[:, 1]))
        .map(partial(af.read_audio_pair,
                     sample_rate=sample_rate),
             num_parallel_calls=n_parallel_readers)
        .map(partial(af.extract_audio_patches_map,
                     fft_hop=fft_hop,
                     patch_window=patch_window,
                     patch_hop=patch_hop,),
             num_parallel_calls=n_parallel_readers)
        .flat_map(af.zip_tensor_slices)
        .map(partial(compute_spectrogram_map,
                     n_fft=n_fft,
                     fft_hop=fft_hop,
                     normalise=normalise),
             num_parallel_calls=n_parallel_readers)
        .shuffle(n_shuffle).batch(batch_size).prefetch(3)
    )

def compute_spectrogram_map(audio_a, audio_b, n_fft, fft_hop, normalise=False):
    spec_a = compute_spectrogram(audio_a, n_fft, fft_hop, normalise, mag_phase=False)
    spec_b = compute_spectrogram(audio_b, n_fft, fft_hop, normalise, mag_phase=False)

    return spec_a, spec_b, audio_a, audio_b

def compute_spectrogram(audio, n_fft, fft_hop, normalise=False, mag_phase=True):
    '''
    Parameters
    ----------
    audio : single to dual channel audio shaped (n_samples, )

    Returns
    -------
    Tensor of shape (n_frames, 1 + n_fft / 2, 2), where the last dimension is (magnitude, phase)
    '''

    def stft(x, normalise, mag_phase):
        spec = librosa.stft(
            x, n_fft=n_fft, hop_length=fft_hop, window='hann')
        
        if mag_phase:
            mag = np.abs(spec)
            if normalise:
                # TODO: normalize?
                mag = (mag - mag.min()) / (mag.max() - mag.min())
            return mag, np.angle(spec)
        else:
            return spec.real, spec.imag

    def mono_func(py_audio, normalise, mag_phase):
        mag, phase = stft(py_audio[:, 0], normalise, mag_phase)
        ret = np.array([mag, phase]).T
        return ret.astype(np.float32)

    with tf.name_scope('read_spectrogram'):
        ret = tf.py_func(mono_func, [audio, normalise, mag_phase], tf.float32, stateful=False)
        ret.set_shape([(audio.get_shape()[0].value/fft_hop) + 1, 1 + n_fft / 2, 2])
    return ret


In [25]:
directory_a = '/home/enterprise.internal.city.ac.uk/acvn728/minimini/Mixed/train-clean-100/19/198/'
directory_b = '/home/enterprise.internal.city.ac.uk/acvn728/minimini/Voice/train-clean-100/19/198/'

#directory_a = 'C:/Users/Toby/MSc_Project/Test_Audio/CHiME/test/Mixed'
#directory_b = 'C:/Users/Toby/MSc_Project/Test_Audio/CHiME/test/Voice'

#directory_a = '/data/Speech_Data/LibriSpeech/Mixed/test-clean/1580/141083/'
#directory_b = '/data/Speech_Data/LibriSpeech/Voice/test-clean/1580/141083/'

#  Create the pipeline
tf.reset_default_graph()
data = dataset.zip_files(directory_a, directory_b)
data = dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise,
                                  mag_phase)

#  Create the iterator
pipe = data.make_initializable_iterator()
mixed_spec, voice_spec, mixed_audio, voice_audio = pipe.get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_spec_trim = mixed_spec[:, :, :-1, :]
voice_spec_trim = voice_spec[:, :, :-1, :]

In [26]:
data

<PrefetchDataset shapes: ((?, 256, 513, 2), (?, 256, 513, 2), (?, 65280, 1), (?, 65280, 1)), types: (tf.float32, tf.float32, tf.float32, tf.float32)>

In [27]:
mixed_spec_trim.shape

TensorShape([Dimension(None), Dimension(256), Dimension(512), Dimension(2)])

In [28]:
class ComplexNumberCapsNet(object):
    
    def __init__(self, mixed_spec, voice_spec, is_training, reuse=True, name='complex_number_capsnet'):
        """
        input_tensor: Tensor with shape [batch_size, height, width, 2], where the two channels are the real 
                      and imaginary parts of the spectrogram
        is_training:  Boolean - should the model be trained on the current input or not
        name:         Model instance name
        """
        with tf.variable_scope(name):
            self.mixed_spec = mixed_spec
            self.voice_spec = voice_spec
            
            with tf.variable_scope('Primary_Caps'):
                
                # Reshape layer to be 1 capsule x [filters] atoms
                _, H, W, C = mixed_spec.get_shape()
                input_caps = layers.Reshape((H.value, W.value, 1, C.value))(mixed_spec)
                self.input_caps = input_caps
            
            with tf.variable_scope('Conv_Caps'):
                conv_caps = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=2, strides=1, padding='same',
                                                               routings=1, name='primarycaps')(input_caps)
                self.conv_caps = conv_caps
                
#            with tf.variable_scope('Seg_Caps'):
#                seg_caps = capsule_layers.ConvCapsuleLayer(kernel_size=1, num_capsule=16, num_atoms=2, strides=1, padding='same',
#                                                           routings=3, name='seg_caps')(conv_caps)
#                self.seg_caps = seg_caps
            
            with tf.variable_scope('Reconstruction'):
                reconstruction = capsule_layers.ConvCapsuleLayer(kernel_size=1, num_capsule=1, num_atoms=2, strides=1, padding='same',
                                                           routings=3, name='seg_caps')(conv_caps)
                reconstruction = layers.Reshape((H.value, W.value, C.value))(reconstruction)
                self.reconstruction = reconstruction
            
            self.cost = mf.l1_loss(self.reconstruction, voice_spec)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=0.0002,
                beta1=0.5,
            )
            self.train_op = self.optimizer.minimize(self.cost)

In [29]:
class ComplexUNetModel(object):
    """
    Top level U-Net object.
    Attributes:
        mixed_mag: Input placeholder for magnitude spectrogram of mixed signals (voice plus background noise) - X
        voice_mag: Input placeholder for magnitude spectrogram of isolated voice signal - Y
        mixed_phase: Input placeholder for phase spectrogram of mixed signals (voice plus background noise)
        mixed_audio: Input placeholder for waveform audio of mixed signals (voice plus background noise)
        voice_audio: Input placeholder for waveform audio of isolated voice signal
        variant: The type of U-Net model (Normal convolutional or capsule based)
        is_training: Boolean - should the model be trained on the current input or not
        name: Model instance name
    """
    def __init__(self, mixed_spec, voice_spec, mixed_audio, voice_audio, variant, is_training, name='complex_unet_model'):
        with tf.variable_scope(name):
    
            self.mixed_spec = mixed_spec
            self.voice_spec = voice_spec
            
            
            self.input_shape = mixed_spec.get_shape().as_list()
            self.mixed_audio = mixed_audio
            self.voice_audio = voice_audio
            self.variant = variant
            self.is_training = is_training

            self.voice_mask_unet = ComplexUNet(mixed_spec, variant, is_training=is_training, reuse=False, name='voice-mask-unet')

            self.voice_mask = self.voice_mask_unet.output

            self.gen_voice = self.voice_mask * mixed_spec

            self.cost = mf.l1_loss(self.gen_voice, voice_spec)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=0.002,
                beta1=0.5,
            )
            self.train_op = self.optimizer.minimize(self.cost)

            
class ComplexUNet(object):

    def __init__(self, input_tensor, variant, is_training, reuse, name):
        with tf.variable_scope(name, reuse=reuse):
            self.variant = variant

            if self.variant == 'unet':
                self.encoder = UNetEncoder(input_tensor, is_training, reuse)
                self.decoder = UNetDecoder(self.encoder.output, self.encoder, is_training, reuse)
            elif self.variant == 'capsunet':
                self.encoder = ComplexEncoder(input_tensor)
                self.decoder = ComplexDecoder(self.encoder.output, self.encoder)

            self.output = mf.tanh(self.decoder.output) / 2 + .5

class ComplexEncoder(object):
    def __init__(self, input_tensor):
        net = input_tensor
        with tf.variable_scope('encoder'):
            with tf.variable_scope('layer-1'):      
                # Reshape layer to be 1 capsule x [filters] atoms
                _, self.H, self.W, self.C = net.get_shape()
                net = layers.Reshape((self.H.value, self.W.value, 1, self.C.value))(net)
                self.l1 = net

            with tf.variable_scope('layer-2'):
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=4, num_atoms=2, strides=2, padding='same',
                                                               routings=3)(net)
                self.l2 = net
            
            with tf.variable_scope('layer-3'):
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=6, strides=2, padding='same',
                                                               routings=3)(net)
                self.l3 = net
            
            with tf.variable_scope('layer-4'):
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=16, num_atoms=12, strides=2, padding='same',
                                                               routings=3)(net)
                
        self.output = net

class ComplexDecoder(object):
    def __init__(self, input_tensor, encoder):
        net = input_tensor
        with tf.variable_scope('decoder'):
            with tf.variable_scope('layer-1'):
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=8, num_atoms=6, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3)(net)
                net = layers.Concatenate(axis=-2)([net, encoder.l3])
                self.l1 = net
                
            with tf.variable_scope('layer-2'):
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=4, num_atoms=2, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3)(net)
                net = layers.Concatenate(axis=-2)([net, encoder.l2])
                self.l2 = net
            
            with tf.variable_scope('layer-3'):
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=1, num_atoms=2, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3)(net)
                #net = layers.Concatenate(axis=-2)([net, encoder.l1])
                self.l3 = net
            #with tf.variable_scope('layer-4'):
            #    net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=1, num_atoms=2, upsamp_type='deconv',
            #                                            scaling=1, padding='same', routings=3)(net)
                
                net = layers.Reshape((encoder.H.value, encoder.W.value, encoder.C.value))(net)
                
            self.output = net
            

In [30]:
model = ComplexUNetModel(mixed_spec_trim, voice_spec_trim, mixed_audio, voice_audio, 'capsunet', is_training=True)

In [31]:
print(model.voice_mask_unet.encoder.l1.get_shape().as_list(), '\n',
      model.voice_mask_unet.encoder.l2.get_shape().as_list(), '\n',
      model.voice_mask_unet.encoder.l3.get_shape().as_list(), '\n',
      model.voice_mask_unet.encoder.output.get_shape().as_list(), '\n',
      model.voice_mask_unet.decoder.l1.get_shape().as_list(), '\n',
      model.voice_mask_unet.decoder.l2.get_shape().as_list(), '\n',
      model.voice_mask_unet.decoder.l3.get_shape().as_list(), '\n',
      #model.voice_mask_unet.decoder.output.get_shape().as_list(), '\n',
      model.gen_voice.get_shape().as_list()
     )

[None, 256, 512, 1, 2] 
 [None, 128, 256, 4, 2] 
 [None, 64, 128, 8, 6] 
 [None, 32, 64, 16, 12] 
 [None, 64, 128, 16, 6] 
 [None, 128, 256, 8, 2] 
 [None, 256, 512, 1, 2] 
 [None, 256, 512, 2]


In [32]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.visible_device_list = str(1)
sess = tf.Session(config=tf_config)
sess.run(tf.global_variables_initializer())
sess.run(pipe.initializer)
print('start: {}'.format(datetime.datetime.now()))
for i in range(1):
    _, mix, voice, est, cost = sess.run([model.train_op, tf.complex(model.mixed_spec[:,:,:,0], model.mixed_spec[:,:,:,1]), 
                                         tf.complex(model.voice_spec[:,:,:,0], model.voice_spec[:,:,:,1]),
                                         tf.complex(model.gen_voice[:,:,:,0], model.gen_voice[:,:,:,0]), 
                                         model.cost])
    
    print('Iteration {i}: {dt}\nCost: {c}'.format(i=i, dt=datetime.datetime.now(), c=cost))

start: 2018-10-23 20:41:00.943889
Iteration 0: 2018-10-23 20:41:02.106541
Cost: 0.2538094222545624


In [19]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.visible_device_list = str(1)
sess = tf.Session(config=tf_config)
sess.run(pipe.initializer)

In [33]:
print('start: {}'.format(datetime.datetime.now()))
epochs = 1000
epoch = 0
epoch_cost = []
i = 0
while epoch < epochs:
    try:
        _, mix, voice, est, cost = sess.run([model.train_op, tf.complex(model.mixed_spec[:,:,:,0], model.mixed_spec[:,:,:,1]), 
                                             tf.complex(model.voice_spec[:,:,:,0], model.voice_spec[:,:,:,1]),
                                             tf.complex(model.gen_voice[:,:,:,0], model.gen_voice[:,:,:,1]), 
                                             model.cost])

        #print('Iteration {i}: {dt}\nCost: {c}'.format(i=i, dt=datetime.datetime.now(), c=cost))
        epoch_cost.append(cost)
        i += 1
    except tf.errors.OutOfRangeError:
        epoch_mean_cost = sum(epoch_cost)/len(epoch_cost)
        print('Epoch {e} finished. Mean cost = {emc}'.format(e=epoch, emc=epoch_mean_cost))
        epoch += 1
        sess.run(pipe.initializer)
print('finish: {}'.format(datetime.datetime.now()))

start: 2018-10-23 20:41:12.834242
Epoch 0 finished. Mean cost = 0.21380672231316566
Epoch 1 finished. Mean cost = 0.2159529510471556
Epoch 2 finished. Mean cost = 0.21467463139976775
Epoch 3 finished. Mean cost = 0.2092367261648178
Epoch 4 finished. Mean cost = 0.2015508363644282


ResourceExhaustedError: OOM when allocating tensor with shape[1,16,64,128,8,6]
	 [[Node: complex_unet_model/gradients/complex_unet_model/voice-mask-unet/decoder/layer-1/deconv_capsule_layer_1/while/mul_3_grad/mul_1 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](complex_unet_model/gradients/complex_unet_model/voice-mask-unet/decoder/layer-1/deconv_capsule_layer_1/while/mul_3_grad/mul_1/Enter, complex_unet_model/gradients/complex_unet_model/voice-mask-unet/decoder/layer-1/deconv_capsule_layer_1/while/Sum_1_grad/Tile)]]

Caused by op 'complex_unet_model/gradients/complex_unet_model/voice-mask-unet/decoder/layer-1/deconv_capsule_layer_1/while/mul_3_grad/mul_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/enterprise.internal.city.ac.uk/acvn728/.local/lib/python3.5/site-packages/ipykernel_launcher.py"

start: 2018-10-23 16:38:47.012817
Epoch 0 finished. Mean cost = 1.8345006878917047e-05
Epoch 1 finished. Mean cost = 1.8275394370216324e-05
Epoch 2 finished. Mean cost = 1.82527096758595e-05
Epoch 3 finished. Mean cost = 1.8241398160024242e-05
Epoch 4 finished. Mean cost = 1.8234578573678698e-05
Epoch 5 finished. Mean cost = 1.8229987729884192e-05
Epoch 6 finished. Mean cost = 1.8226663270206178e-05
Epoch 7 finished. Mean cost = 1.822412628716299e-05
Epoch 8 finished. Mean cost = 1.8222111772761957e-05
Epoch 9 finished. Mean cost = 1.822046098720921e-05
finish: 2018-10-23 16:44:09.720241


In [61]:
voice_est_wave = list()
voice_wave = list()
mix_wave = list()


for i in range(est.shape[0]):
    voice_est_wave.append(librosa.istft(est[i, :, :].T, fft_hop))

for i in range(voice.shape[0]):
    voice_wave.append(librosa.istft(voice[i, :, :].T, fft_hop))
    
for i in range(mix.shape[0]):
    mix_wave.append(librosa.istft(mix[i, :, :].T, fft_hop))

In [62]:
ipd.Audio(voice_est_wave[0].T, rate=sample_rate)

In [63]:
ipd.Audio(voice_wave[0].T, rate=sample_rate)

In [64]:
ipd.Audio(mix_wave[0], rate=sample_rate)

In [ ]:
ipd.Audio(voice_est_wave[0].T, rate=sample_rate)

In [ ]:
spec = librosa.stft(voice_est_wave[0], n_fft=n_fft, hop_length=fft_hop, window='hann')
mag = np.abs(spec)
plt.imshow(mag[:125,:])